In [ ]:
from cognee import add, cognify, search
from cognee.utils import render_graph
from os import listdir, path

data_path = path.abspath("../.data")

print(data_path)

await add(path.abspath("../.data"), "izmene")

graph = await cognify("izmene")

await render_graph(graph, graph_type="networkx")